In [2]:
import pandas as pd
import sys
from pickle import dump,load
sys.path.append('../../')
%load_ext autoreload

In [27]:
df=pd.read_csv('../data/modified/data_to_work_on.csv')

In [5]:
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [105]:
%autoreload 2
from no_toxicity.src.preprocessing.cleaning import Dfcleaner

In [47]:
# Making the list of column to clean
raw_comments=df['comment_text'].to_list()

In [106]:
cleaner=Dfcleaner()


In [22]:
cleaned_comments=cleaner.clean(raw_comments,stem=False)

10000 examples cleaned out of 159571
20000 examples cleaned out of 159571
30000 examples cleaned out of 159571
40000 examples cleaned out of 159571
50000 examples cleaned out of 159571
60000 examples cleaned out of 159571
70000 examples cleaned out of 159571
80000 examples cleaned out of 159571
90000 examples cleaned out of 159571
100000 examples cleaned out of 159571
110000 examples cleaned out of 159571
120000 examples cleaned out of 159571
130000 examples cleaned out of 159571
140000 examples cleaned out of 159571
150000 examples cleaned out of 159571
Cleaning Done


In [129]:
cleaned_comments=cleaner.remove_frequent_rare(cleaned_comments,frequent=True,rare=True)

10000 examples cleaned out of 159517
20000 examples cleaned out of 159517
30000 examples cleaned out of 159517
40000 examples cleaned out of 159517
50000 examples cleaned out of 159517
60000 examples cleaned out of 159517
70000 examples cleaned out of 159517
80000 examples cleaned out of 159517
90000 examples cleaned out of 159517
100000 examples cleaned out of 159517
110000 examples cleaned out of 159517
120000 examples cleaned out of 159517
130000 examples cleaned out of 159517
140000 examples cleaned out of 159517
150000 examples cleaned out of 159517
Cleaning Done


In [131]:
# Replacing with new data
df['comment_text']=cleaned_comments

In [133]:
# # Saving the cleaned data
# df.to_csv('../data/interim/cleaned_data1.csv',index=False)

In [3]:
# Load the data
df=pd.read_csv('../data/interim/cleaned_data1.csv')

In [4]:
df.dropna(inplace=True)

In [5]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation edits username hardcore metallica ...,0,0,0,0,0,0
1,aww match background colour seemingly stuck th...,0,0,0,0,0,0
2,hey man really try war guy constantly remove r...,0,0,0,0,0,0
3,real suggestion improvement wonder section sta...,0,0,0,0,0,0
4,sir hero chance remember,0,0,0,0,0,0


In [6]:
def new_col_value(row):
    flag=True
    # print(row.iloc[1:])
    for i in row.iloc[1:]:
        if i==1:
            flag=False
            break
    if flag:
        return 1
    else:
        return 0


In [7]:
df['not_toxic']=df.apply(lambda row:new_col_value(row),axis=1)

In [8]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,not_toxic
0,explanation edits username hardcore metallica ...,0,0,0,0,0,0,1
1,aww match background colour seemingly stuck th...,0,0,0,0,0,0,1
2,hey man really try war guy constantly remove r...,0,0,0,0,0,0,1
3,real suggestion improvement wonder section sta...,0,0,0,0,0,0,1
4,sir hero chance remember,0,0,0,0,0,0,1


In [9]:
df.iloc[:,1:].sum()

toxic             15294
severe_toxic       1595
obscene            8449
threat              478
insult             7877
identity_hate      1405
not_toxic        143240
dtype: int64

In [10]:
from imblearn.under_sampling import RandomUnderSampler

In [17]:
rus = RandomUnderSampler(random_state=42)
df,Y= rus.fit_resample(df.iloc[:,:-1],df.iloc[:,-1])

In [18]:
df['not_toxic']=Y

In [19]:
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,not_toxic
0,cocksucker piss around work,1,1,1,0,1,0,0
1,hey exclusive group wp taliban good destroy se...,1,0,0,0,0,0,0
2,bye look come think comming back tosser,1,0,0,0,0,0,0
3,gay antisemmitian archangel white tiger meow g...,1,0,1,0,1,1,0
4,fuck filthy mother as dry,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...
32445,question think troll could explain nearly edit...,0,0,0,0,0,0,1
32446,hello kyo sorry hear busy hope today find well...,0,0,0,0,0,0,1
32447,zeer oude klanken en heel nieuwe geluiden surp...,0,0,0,0,0,0,1
32448,response tord departure eddsworld follow death...,0,0,0,0,0,0,1


In [23]:
%autoreload 2
from no_toxicity.src.preprocessing.preprocess import PrepareTheData

In [33]:
data=PrepareTheData(df[['comment_text']],df.iloc[:,1:])

../..\no_toxicity\src\preprocessing\preprocess.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_text'] = df.comment_text.apply(tokenize)


Tokenization Done


C:\Users\dummy\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Replacing Done
token2idx created
tokens_to_indices created
Presprocessing done


../..\no_toxicity\src\preprocessing\preprocess.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['common_text'] = df.common_text.apply(


In [34]:
data.__getitem__(0)

KeyError: 0

In [30]:
df[['comment_text']].commen

,comment_text
0,cocksucker piss around work
1,hey exclusive group wp taliban good destroy se...
2,bye look come think comming back tosser
3,gay antisemmitian archangel white tiger meow g...
4,fuck filthy mother as dry
...,...
32445,question think troll could explain nearly edit...
32446,hello kyo sorry hear busy hope today find well...
32447,zeer oude klanken en heel nieuwe geluiden surp...
32448,response tord departure eddsworld follow death...
